# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [0]:
# 打開瀏覽器
# !pip install selenium
# !apt install chromium-chromedriver

import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('chromedriver',options=options)
browser.maximize_window()

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [0]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
# 參考Stack Overflow:  https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
#

browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 0.001

# Get scroll height.
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    print('網頁更新中...')

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print('更新完成')

        break

    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
更新完成


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [0]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#

def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 父欠債建中生被逼下跪！他目睹債主將刀捅進書桌...發憤學投資翻身成億元大戶

▲傅君玉的投資座右銘「浮光躍金、靜影沉璧」，意思是繁華世界攪動人心，在股海裡心則要平靜。
圖文／鏡週刊
存股大戶傅君玉為何鍾情有土地資產、股息牢靠的公司，且積極將領到的股息再投入房地產市場？他坦言，或許與年少時期家中境遇有關，「母親說房子能遮頭（意味有依靠），這對以前家裡付不出房租、長年沒有安全感的我來說，可能有點彌補作用。」
「我從小就很會讀書，念的學校都是第一志願，有一天放學回家，一位陌生男子大搖大擺坐在我家沙發上，母親二話不說，竟劈頭就要我向陌生人下跪。」傅君玉說，「我是個驕傲的建中生，怎麼可能隨便跟人低頭；於是轉身就進房間，母親還追上來作勢要打我。」
「為了躲債主，我寄宿同學家，我沒向債主下跪磕頭，母親倒是向同學的父母下跪了。老實說當時太多的不堪，對日後身心都有巨大影響，像債主威脅家人，將刀捅進我的書桌，那把刀，至少插在我心中30年揮之不去。」
年少時因為父親不懂英文合約，做生意虧損、債務愈滾愈大；基於安全考量，傅君玉與弟弟被迫寄人籬下，為償還家中債務，姐姐甚至差點下海當舞女，幸虧在緊要關頭，考上華航空姐，優渥的薪資待遇，才得以讓傅家喘息。只是往事並不如煙，自尊與自卑，全都化為傅君玉拚命賺錢的動力。
靠存股擺脫過往命運，傅君玉甚至翻轉成資產數億元的大戶，問他如今台股高掛，還能存股嗎？他說，「很多人會有指數迷思，以為現在是高點，投資應當保守，其實每年台股除息蒸發掉好幾百點，歷史高點12682早就突破了；況且，隨著上市公司增加，指數本來就會逐步墊高，看指數位置一點意義也沒有！」傅君玉直言，像我的股市資產就比12682點當時多了50至60倍，這又怎麼解釋。
投資先要破除迷思、不該自我設限，才可能壯大財富，這是傅君玉給投資人最珍貴的提點。
----------------------------------------------------------------------

[1] 哈日族衝了！日圓改寫1年來新低　換新台幣5萬賺到2張迪士尼門票

▲ 東京迪士尼樂園。（圖／記者蔡玟君攝）
記者李瑞瑾／台北報導
新台幣強勢無法擋，也讓日